## Hello, Here's How to use RAG w HF Models

Install some dependencies

In [ ]:
!pip install -q -U peft==0.8.2
!pip install -q -U trl==0.7.10
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.1
!pip install langchain sentence-transformers chromadb langchainhub

!pip install langchain-community langchain-core

Get the Model You Want

In [3]:
#!pip install --upgrade --upgrade-strategy eager "optimum[neural-compressor]"


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb

import time
import uuid
#from fuzzywuzzy import fuzz
#from optimum.quantization import QuantizeruenceClassification


Define Variables

In [ ]:
import os
import torch

# set your own hf token then fetch it here

hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token


# model.half() -- gpu
model.to(dtype=torch.float16) # cpu

Define Data Sources

In [9]:
import pandas as pd

file_names = [
    "study_permit_general",
    "work_permit_student_general",
    "work-study-data-llm",
    "vancouver_transit_qa_pairs",
    "permanent_residence_student_general",
    "data-with-sources"
]

all_texts = []

for file in file_names:
    path = f'./sample_data/{file}.csv'
    try:
        df = pd.read_csv(path)
        df.columns = df.columns.str.lower()

        if 'question' in df.columns and 'answer' in df.columns:
            df = df.drop_duplicates(subset=['question'])
            df['text'] = df['question'].fillna('') + ' ' + df['answer'].fillna('')
        else:
            print(f"no text columns in {file}")
            continue
        all_texts.extend(df['text'].tolist())
    except Exception as e:
        print(f"Error loading {file}: {e}")


Set Embedding Model, and Chroma Client to Interact w Vector Database and Create Collections

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import uuid
import pandas as pd
#from fuzzywuzzy import fuzz

# pt model for generating embeddings used pretty often
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# persistent client to interact w chroma vector store
client = chromadb.PersistentClient(path="./chroma_db")

# create collections for each data (for testing rn)
collection = client.get_or_create_collection(name="combined_docs")

# seems like better results if we remove duplicates and very similar data
data = pd.DataFrame({"text": all_texts})
data = data.drop_duplicates()
all_texts = data["text"].tolist()

print(f"successfully added {len(all_texts)} documents to Chroma DB.")

<ipython-input-10-7b0600c45c22>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


successfully added 1031 documents to Chroma DB.


Function to add data to collection by embedding them

In [12]:
def add_data_to_collection_batch(collection, texts, batch_size=3):
    for idx in range(0, len(texts), batch_size):
        try:
            batch_texts = texts[idx: idx + batch_size]

            embeddings = embedding_model.embed_documents(batch_texts)

            batch_ids = [str(uuid.uuid4()) for _ in batch_texts]

            collection.add(
                ids=batch_ids,
                embeddings=embeddings,
                documents=batch_texts
            )
            print(f"successfully added {len(batch_texts)} documents (Batch {idx}-{idx + batch_size - 1})")
        except Exception as e:
            print(f"Error processing batch starting at index {idx}: {e}")

In [ ]:
add_data_to_collection_batch(collection, all_texts)
print(f"successfully added {len(all_texts)} documents to the Chroma collection.")

Function to now match for releveant document

In [14]:
def get_relevant_documents(query, n_results=3):
    try:
        query_embeddings = embedding_model.embed_documents([query])[0]

        results = collection.query(query_embeddings=[query_embeddings], n_results=n_results)
        print(f"Query Results: {results}")

        return results['documents'][0] if results['documents'] else []
    except Exception as e:
        print(f"Error querying: {e}")
        return []

Generate Answer

In [22]:
def generate_answer(query):
    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=512)
    base_output = model.generate(inputs["input_ids"], temperature=0.1, max_new_tokens=200, eos_token_id=model.config.eos_token_id)
    response_before_rag = tokenizer.decode(base_output[0], skip_special_tokens=True)

    relevant_documents = get_relevant_documents(query)
    if not relevant_documents:
        return {
            "Before RAG Response": response_before_rag,
            "After RAG Response": "Sorry, no relevant documents found."
        }

    relevant_texts = "\n\n".join([doc for doc in relevant_documents])
    rag_prompt = f"""
    You are a helpful assistant for international students. Here are relevant documents:

    {relevant_texts}

    Please respond to the following question based on the documents above. Be conversational but concise:

    Question: {query}

    Answer:
    """

    rag_inputs = tokenizer(rag_prompt, return_tensors="pt", truncation=True, padding=True, max_length=512)
    rag_output = model.generate(rag_inputs["input_ids"], temperature=0.1, max_new_tokens=300, eos_token_id=model.config.eos_token_id)
    response_after_rag = tokenizer.decode(rag_output[0], skip_special_tokens=True)

    return {
        "Before RAG Response": response_before_rag,
        "After RAG Response": response_after_rag
    }


Example Usage

In [23]:
test_queries = [
    "How do I apply for a study permit in Canada?",
    "Can I work while studying on a student visa?",
    "What happens if my study permit expires before I finish my program?",
    "Do I need a new study permit if I change schools?",
    "How long does it take to process a Canadian study permit?",
    "Am I allowed to work off-campus as an international student?",
    "How many hours can I work while studying in Canada?",
    "What documents do I need to apply for a co-op work permit?",
    "Can I work in Canada after I graduate?",
    "What is a Post-Graduation Work Permit (PGWP) and how do I apply?",
    "How do I apply for MSP (Medical Services Plan) in British Columbia?",
    "Is MSP mandatory for international students?",
    "What healthcare services are covered under MSP?",
    "What should I do if I get sick and don’t have insurance yet?",
    "Can I use private health insurance instead of MSP?",
    "What are my options for student housing in Vancouver?",
    "How much does rent typically cost for international students?",
    "What should I check before signing a lease in Canada?",
    "Are there any student discounts for accommodation?",
    "How can I find a roommate in Canada?",
    "How do I open a bank account as an international student?",
    "What documents do I need to get a student bank account?",
    "Can I get a credit card as an international student?",
    "How do I send money to my home country from Canada?",
    "What scholarships are available for international students?",
    "How does the Compass Card work for transit in Vancouver?",
    "Am I eligible for a U-Pass as an international student?",
    "What is the best way to get around Vancouver on a budget?",
    "Where can I find the bus and SkyTrain schedules?",
    "Are there student discounts for public transportation?",
    "Can I apply for permanent residence after graduating?",
    "What is the Canadian Experience Class (CEC) immigration program?",
    "How can I improve my chances of getting permanent residence?",
    "What are the eligibility requirements for Express Entry?",
    "Does having a Canadian degree help with PR applications?"
]

for idx, user_query in enumerate(test_queries, start=1):
    responses = generate_answer(user_query)

    print("\n" + "="*50)
    print(f"Test Query {idx}: {user_query}")
    print("="*50)
    print("Response Before RAG:\n", responses["Before RAG Response"])
    print("\nResponse After RAG:\n", responses["After RAG Response"])
    print("="*50 + "\n\n")

#add_all_texts_to_collection()

# takes 1 min 15 seconds ish to get relevant documents
#


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query Results: {'ids': [['587d3202-cbc3-466d-89f0-3e85fca2e893', 'a49ccbb2-4727-4152-b7a5-a42c592873bd', 'b55f7a0d-1f7f-44ca-9e17-50cdb7b7348c']], 'embeddings': None, 'documents': [['How do I pay for a study permit?("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=481&top=15") You must pay a processing fee with your application. For details, visit the\xa0Pay your fees page("https://ircc.canada.ca/english/information/fees/index.asp")\xa0to learn about the methods of payment accepted by Canadian visa offices.', 'How do I pay for a study permit?("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=481&top=15") You must pay a processing fee with your application. For details, visit the\xa0Pay your fees page("https://ircc.canada.ca/english/information/fees/index.asp")\xa0to learn about the methods of payment accepted by Canadian visa offices.', 'Do I need a permit to study in Canada?("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=478&top=15") Most\xa0foreign nati

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Test Query 1: How do I apply for a study permit in Canada?
Response Before RAG:
 How do I apply for a study permit in Canada? The application process typically involves several steps, which I'll outline below.

## Step 1: Determine Your Eligibility
To apply for a study permit in Canada, you must meet certain eligibility criteria. These include being a permanent resident of Canada, having a valid passport, and having sufficient funds to support yourself during your stay in Canada.

## Step 2: Choose the Correct Study Permit Type
There are several types of study permits available, including a study permit for a specific program, a study permit for a specific province, and a study permit for a specific field of study. You must choose the correct type of permit based on your program and province of study.

## Step 3: Gather Required Documents
To apply for a study permit, you will need to gather several documents, including:
- A valid passport
- Proof of permanent residency in Canada
- Pro

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query Results: {'ids': [['84bf7916-fdb2-40b0-99a8-ea473684f0b7', '3a87dfbb-8e45-4785-aca5-b51c2abc5ff2', '95b219c8-8481-417c-a00f-d0312fdb38d7']], 'embeddings': None, 'documents': [['What if my school is no longer on the designated learning institution (DLI) list? You can either keep studying until your current permit expires or transfer to another DLI.', 'What if my school is no longer on the designated learning institution (DLI) list? You can either keep studying until your current permit expires or transfer to another DLI.', 'What if my study permit has been approved but not issued yet? If your study permit is approved but not issued, you’ll need to provide a new letter of acceptance from a new DLI and update your information in MyAccount.']], 'uris': None, 'data': None, 'metadatas': [[None, None, None]], 'distances': [[0.6232689619064331, 0.6232689619064331, 0.7482203841209412]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.m

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Test Query 4: Do I need a new study permit if I change schools?
Response Before RAG:
 Do I need a new study permit if I change schools??
If you are a Canadian citizen or permanent resident, you may need to apply for a new study permit if you change schools. Here's what you need to know:

**Reasons for a new study permit:**

1. **School change:** If you change schools, you may need to apply for a new study permit. This is because your study permit is tied to your current school.
2. **Changes in your personal details:** If you change your name, address, or other personal details, you may need to apply for a new study permit.
3. **Changes in your immigration status:** If you change your immigration status, such as from a student visa to a work permit, you may need to apply for a new study permit.

**How to apply for a new study permit:**

1. **Check if you need a new study permit:** Visit the Immigration, Refugees and Citizenship Canada (IRCC) website to see if you need a new study permi

KeyboardInterrupt: 